# Calculate LD
- get token from https://ldlink.nci.nih.gov/?tab=apiaccess
- token: 44c6c4744e3e

In [2]:
library(seqminer)
library(LDlinkR)

In [3]:
token = "44c6c4744e3e"

In [4]:
setwd("/frazer01/projects/CARDIPS/analysis/cardiac_gwas_coloc")

source("script/functions.R"  )


In [5]:
dir.create("pipeline/5.7.ld_for_figures", showWarnings = FALSE)


In [6]:
snps = fread(cmd = "zcat /frazer01/reference/public/ukbb/variants.tsv.bgz", sep = "\t", header = TRUE, data.table = FALSE)
snps$chrom = as.numeric(snps$chr)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [20]:
calculate_ld = function(signal_id, variant_id, distance = 100000, pop = "CEU")
{
    indata     = readRDS(paste0("pipeline/5.5.meta_analysis/coloc_data/", signal_id, ".rds"))
    meta       = indata[["meta_analysis_gwas"]]
    meta$index = FALSE
    meta$pos   = as.numeric(unlist(lapply(meta$id, function(x){unlist(strsplit(x, "_"))[[3]]})))
    
    meta[meta$id == variant_id, "index"] = TRUE
    
    meta          = meta[meta$pos >= meta[meta$index == TRUE, "pos"] - distance & meta$pos <= meta[meta$index == TRUE, "pos"] + distance,]
    #meta$distance = abs(meta$pos - meta[meta$index == TRUE, "pos"])
    #
    #if(nrow(meta) > 1000)
    #{
    #    meta = meta[order(meta$distance),]
    #    meta = meta[1:1000,]
    #}
    
    meta$chrom = as.numeric(unlist(strsplit(meta[meta$index == TRUE, "id"], "_"))[[2]])
    #meta       = merge(meta, snps[,c("chrom", "pos", "rsid")], by = c("chrom", "pos"), all.x = TRUE)
    rsid       = snps[snps$chrom == meta[meta$index == TRUE, "chrom"] & snps$pos == meta[meta$index == TRUE, "pos"], "rsid"][[1]]

    my_proxies = LDproxy(snp = rsid, pop = pop, r2d = "r2", token = token)
    
    my_proxies$chrom = as.numeric(sub("chr", "", unlist(lapply(my_proxies$Coord, function(coord){unlist(strsplit(coord, ":"))[[1]]}))))
    my_proxies$pos   = as.numeric(               unlist(lapply(my_proxies$Coord, function(coord){unlist(strsplit(coord, ":"))[[2]]})))
    my_proxies       = data.frame(rsid = my_proxies$RS_Number, r2 = my_proxies$R2, pos = my_proxies$pos, chrom = my_proxies$chrom)
    
    meta = merge(meta, my_proxies, by = c("chrom", "pos"), all.x = TRUE)
    
    return(meta)
}

rs13107325 = calculate_ld("4_101904130_104478012.1", "VAR_4_103188709_C_T", 100000, "CEU")
rs1229984  = calculate_ld("4_99841138_100760909.1" , "VAR_4_100239319_T_C", 100000, "EAS")

		


LDlink server is working...



LDlink server is working...




In [22]:
fwrite(rs13107325, "pipeline/5.7.ld_for_figures/rs13107325.txt", sep = "\t", col.names = TRUE, row.names = FALSE)
fwrite(rs1229984 , "pipeline/5.7.ld_for_figures/rs1229984.txt" , sep = "\t", col.names = TRUE, row.names = FALSE)
